# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Convert visit_date to datetime
df['visit_date'] = pd.to_datetime(df['visit_date'])

# Filter for July 2024
july_data = df[
    (df['visit_date'].dt.year == 2024) &
    (df['visit_date'].dt.month == 7)
]

# Calculate spending per guest per visit per experience
per_visit_spending = july_data.groupby(
    ['guest_id', 'visit_date', 'park_experience_type']
)['amount_spent'].sum().reset_index()

# Calculate average per-visit spending per experience type
avg_spending_by_experience = per_visit_spending.groupby(
    'park_experience_type'
)['amount_spent'].mean().reset_index()

# Rename column for clarity
avg_spending_by_experience.rename(columns={'amount_spent': 'avg_spending_per_visit'}, inplace=True)

# Include experience types with no transactions
all_experience_types = df['park_experience_type'].dropna().unique()
all_experience_types_df = pd.DataFrame({'park_experience_type': all_experience_types})

# Join to ensure all experience types are included
final_result = pd.merge(all_experience_types_df, avg_spending_by_experience,
                        on='park_experience_type', how='left')

# Fill missing values (no spending in July) with 0.0
final_result['avg_spending_per_visit'] = final_result['avg_spending_per_visit'].fillna(0.0)

# Sort
final_result = final_result.sort_values(by='avg_spending_per_visit', ascending=False)

# Show result
print(final_result)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Convert visit_date to datetime
df['visit_date'] = pd.to_datetime(df['visit_date'])

# Filter for August 2024
august_data = df[
    (df['visit_date'].dt.year == 2024) &
    (df['visit_date'].dt.month == 8)
]

# Sum spending per guest per visit
visit_spending = august_data.groupby(['guest_id', 'visit_date'])['amount_spent'].sum().reset_index()

# Keep only guests with more than one visit
visit_counts = visit_spending['guest_id'].value_counts()
multi_visit_guests = visit_counts[visit_counts > 1].index
multi_visit_data = visit_spending[visit_spending['guest_id'].isin(multi_visit_guests)]

# For each guest, get first and last visit spending
first_last_spending = multi_visit_data.sort_values(['guest_id', 'visit_date']).groupby('guest_id').agg(
    first_visit_spending=('amount_spent', 'first'),
    last_visit_spending=('amount_spent', 'last')
).reset_index()

# Calculate the difference
first_last_spending['spending_difference'] = (
    first_last_spending['last_visit_spending'] - first_last_spending['first_visit_spending']
)

# Show result
print(first_last_spending)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Convert visit_date to datetime
df['visit_date'] = pd.to_datetime(df['visit_date'])

# Filter for September 2024
sept_data = df[
    (df['visit_date'].dt.year == 2024) &
    (df['visit_date'].dt.month == 9)
]

# Group by guest and calculate total spending
total_spending = sept_data.groupby('guest_id')['amount_spent'].sum().reset_index()

# Exclude guests with no purchases
total_spending = total_spending[total_spending['amount_spent'] > 0]

# Define segmentation function
def categorize_spending(amount):
    if amount < 50:
        return 'Low'
    elif amount < 100:
        return 'Medium'
    else:
        return 'High'

# Apply segmentation
total_spending['spending_segment'] = total_spending['amount_spent'].apply(categorize_spending)

# Show result
print(total_spending)

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)